In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample data of movies
movies_data = pd.DataFrame({
    'MovieID': [1, 2, 3, 4, 5, 6],
    'Title': ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E', 'Movie F'],
    'Genre': ['Action', 'Comedy', 'Action|Thriller', 'Comedy|Romance', 'Thriller', 'Drama']
})

# Sample data of users and their liked movies
users_data = pd.DataFrame({
    'UserID': [1, 2],
    'LikedMovies': [['Movie A', 'Movie C'], ['Movie B', 'Movie D']]
})

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_data['Genre'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(user_likes, cosine_sim, movies_data):
    movie_indices = []
    for movie_title in user_likes:
        movie_indices.append(movies_data.index[movies_data['Title'] == movie_title].tolist()[0])
    sim_scores = cosine_sim[movie_indices].sum(axis=0)
    recommended_indices = sim_scores.argsort()[::-1]
    recommended_indices = [idx for idx in recommended_indices if idx not in movie_indices]
    recommended_movies = movies_data[['Title', 'Genre']].iloc[recommended_indices]

    return recommended_movies

def get_user_recommendations(user_id, cosine_sim, movies_data, users_data):
    user_likes = users_data.loc[users_data['UserID'] == user_id, 'LikedMovies'].iloc[0]
    recommendations = get_recommendations(user_likes, cosine_sim, movies_data)
    most_recommended_genre = recommendations['Genre'].mode().iloc[0]
    most_recommended_title = recommendations['Title'].iloc[0]
    return recommendations, most_recommended_genre, most_recommended_title

# Get recommendations for each user
for user_id in users_data['UserID']:
    user_recommendations, most_recommended_genre, most_recommended_title = get_user_recommendations(user_id, cosine_sim, movies_data, users_data)
    print(f"\nRecommended Movies for User {user_id}:")
    print(user_recommendations)
    print(f"\nMost Recommended Genre: {most_recommended_genre}")
    print(f"\nMost Recommended Movie Title: {most_recommended_title}")
    print("\n")




Recommended Movies for User 1:
     Title           Genre
4  Movie E        Thriller
5  Movie F           Drama
3  Movie D  Comedy|Romance
1  Movie B          Comedy

Most Recommended Genre: Comedy

Most Recommended Movie Title: Movie E



Recommended Movies for User 2:
     Title            Genre
5  Movie F            Drama
4  Movie E         Thriller
2  Movie C  Action|Thriller
0  Movie A           Action

Most Recommended Genre: Action

Most Recommended Movie Title: Movie F


